# Ensemble Models

-- It's better to use models that are essentially different, as the weakness in one balances the other one.

-- If all models can estimate class probability, then you can set *voting=soft*. This often performs better because it gives more weight to confident scores.

-- Another **way to ensemble models is to train the same model with different samples from you training data**. Here we have *bagging (bootstrap aggregating, with replacament)* and *pasting (without replacement)*. Bagging often performs better, so it's preferred. They scale well because they can run in parallel.

-- **Random Patches Method**: Sample both training instances and features.

-- **Random Subspaces Method**: Sample features, keeps all training instances (i.e., bootstrap=False and max_samples=1.0)

-- The aggregation function is typically the mode (most frequent value) or the average (for regression) // HARD voting

-- Random Forests are great to examine Feature Importance, specially when you need to perform feature selection.

-- **Boosting** is taking several weak learners models, in a sequence, and improving their performance. Every new model has a small improvement from the model before it.
It does not scale well since it can't always run in parallel.  GradientBoosting is a boosting ensemble model, that picks up on the residual errors of the trees, in sequence. In contrary, ADA tweaks the weights after evert iteration.
A learning rate of 0.1 is low, but it usually learns well.

-- A good way to deal with a model taking a long time to learn (specially boosting with lower learning rates) is to perform **early stop**

-- If you set *warm_start=True*, Scikit learn keeps the existing trees when .fit is called, allowing incremental training.

-- Use XGBoost for scalability, portability

-- **Stacking** is when we train a model to perform the aggregation of values. Each model in the sequence generates an output. These outputs are combined for a final one (called a blender, or meta learner)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

logreg = LogisticRegression()
forest = RandomForestClassifier()
svm = SVC()

voting_classifier = VotingClassifier(
    estimators=[("linear", logreg), ("forest", forest), ("svm", svm)],
    voting="hard"
)

# fit the classifier
voting_classifier.fit(X_train,y_train)

# check each classifier accuracy on the test set
from sklearn.metrics import accuracy_score

for clf in (logreg, forest, svm, voting_classifier):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(score.__class__.__name__, accuracy_score(y_pred, y_test))

In [ ]:
# Bagging and Pasting in Scikit-Learn

# the code below will take a X_train and use only 100 samples for each training
# it will probably generalise it better than a single decision tree

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_classifier = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, # if you change boostrap=False then you have Pasting, and not longer Bagging
    # n_jobs=-1 uses all the cores available
    oob_score= True # if you set this to true, you can use the score of "out of bag" as an evaluation, since these instances were not used to train the model
)

bagging_classifier.fit(X_train, y_train)

y_pred = bag_classifier.predict(X_test)

# if you set oob_score=True you can get the score with
bagging_classifier.oob_score_
# you can also check the decision function
bagging_classifier.obb_decision_function_
# it will show for example, the probability of belonging to one class or another

## REVIEW

-- If you have trained five different models and they all achieve 95% precision, you can try combining them into a voting ensemble, which will often give you even better results. It works better if the models are very different (e.g., an SVM classifier, a Decision Tree classifier, a Logistic Regression classifier, and so on). It is even better if they are trained on different training instances (that's the whole point of bagging and pasting ensembles), but if not this will still be effective as long as the models are very different.

-- **A hard voting classifier just counts the votes of each classifier in the ensemble and picks the class that gets the most votes. A soft voting classifier computes the average estimated class probability for each class and picks the class with the highest probability.** This gives high-confidence votes more weight and often performs better, but it works only if every classifier is able to estimate class probabilities (e.g., for the SVM classifiers in Scikit-Learn you must set probability=True).

-- It is quite possible to **speed up training of a bagging ensemble by distributing it across multiple servers**,** since each predictor in the ensemble is independent of the others. The same goes for pasting ensembles and Random Forests, for the same reason. However, each predictor in a boosting ensemble is built based on the previous predictor, so training is necessarily sequential, and you will not gain anything by distributing training across multiple servers. Regarding stacking ensembles, all the predictors in a given layer are independent of each other, so they can be trained in parallel on multiple servers. However, the predictors in one layer can only be trained after the predictors in the previous layer have all been trained.

-- **With out-of-bag evaluation, each predictor in a bagging ensemble is evaluated using instances that it was not trained on (they were held out)**. This makes it possible to have a fairly unbiased evaluation of the ensemble without the need for an additional validation set. Thus, you have more instances available for training, and your ensemble can perform slightly better.

-- When you are growing a tree in a Random Forest, only a random subset of the features is considered for splitting at each node. This is true as well for **Extra-Trees**, but they go one step further: rather than searching for the best possible thresholds, like regular Decision Trees do, they **use random thresholds for each feature**. This extra randomness acts like a form of regularization: if a Random Forest overfits the training data, Extra-Trees might perform better. Moreover, since Extra-Trees don't search for the best possible thresholds, they are much faster to train than Random Forests. However, they are neither faster nor slower than Random Forests when making predictions.

-- If your AdaBoost ensemble underfits the training data, you can try increasing the number of estimators or reducing the regularization hyperparameters of the base estimator. You may also try slightly increasing the learning rate.

-- If your Gradient Boosting ensemble overfits the training set, you should try decreasing the learning rate. You could also use early stopping to find the right number of predictors (you probably have too many).